In [1]:
###############################################################################
# The Institute for the Design of Advanced Energy Systems Integrated Platform
# Framework (IDAES IP) was produced under the DOE Institute for the
# Design of Advanced Energy Systems (IDAES).
#
# Copyright (c) 2018-2023 by the software owners: The Regents of the
# University of California, through Lawrence Berkeley National Laboratory,
# National Technology & Engineering Solutions of Sandia, LLC, Carnegie Mellon
# University, West Virginia University Research Corporation, et al.
# All rights reserved.  Please see the files COPYRIGHT.md and LICENSE.md
# for full copyright and license information.
###############################################################################

# IDAES Model Diagnostics Toolbox Tutorial
Author: Andrew Lee  
Maintainer: Andrew Lee  
Updated: 2023-10-31  

As you have likely discovered already, developing and solving models in an equation-oriented (EO) environment can be challenging and often takes a significant amount of effort. There are many pitfalls and mistakes that can be encountered when developing a model which can greatly impact the solvability and robustness of the final problem.

Model diagnosis and debugging is often more of an art than a science, and it generally relies on significant experience and understanding both of general EO modeling techniques and the specific model and problem being solved. To assist with this process, IDAES has developed a model diagnostics toolbox that brings together a large number of tools for identifying potential issues in a model to help guide the user through the process of finding and resolving these issues. Note however that whilst these tools can help identify the presence of an issue, remedying the issue always requires some degree of engineering knowledge about the system being modeled, and thus it is ultimately up to the user to find a solution to the problem.

This tutorial will take you through using the {py:class}`DiagnosticsToolbox <idaes.core.util.model_diagnostics.DiagnosticsToolbox>` to debug a number of issues in a simple Pyomo model and to take it from initially reporting a possible infeasible solution to returning the correct solution.

To get started, the ``DiagnosticsToolbox`` can be imported from ``idaes.core.util``.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Import the DiagnosticsToolbox in the cell below.
</div>

In [2]:
from idaes.core.util import DiagnosticsToolbox

To get some information on where to start, try using the Python ``help()`` function to see the documentation for the ``DiagnosticsToolbox``.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Call `help(DiagnosticsToolbox)` to see some more information on the toolbox and some instructions on how to get started.
</div>

In [3]:
# Call the help() function for more information

The ``help()`` function gives us a lot of information on the ``DiagnosticsToolbox`` and all the methods that it supports (and there are many). However, the important part to start with are the four steps outlined at the top of the doc string that tell us how to get started.

Firstly, we need a model to test (and, for this tutorial at least, one that has a wide range of issues that we need to fix before it will solve). We then also need to fix some variables so that we have 0 degrees of freedom in our model. Whilst our ultimate goal is generally optimization (and thus a system with 1 or more degrees of freedom), all models conceptually derive from a square model representing a nominal state. If this nominal state is not well-posed, then any issues present will also be present in the resulting optimization (even if adding degrees of freedom means that the model is now easier to solve).

The cell below contains a demonstration model for this tutorial that contains a number of issues that we will resolve using the ``DiagnosticsToolbox``.

In [5]:
import pyomo.environ as pyo

m = pyo.ConcreteModel()

m.v1 = pyo.Var(units=pyo.units.m)
m.v2 = pyo.Var(units=pyo.units.m)
m.v3 = pyo.Var(bounds=(0, 5))
m.v4 = pyo.Var()
m.v5 = pyo.Var(bounds=(0, 10))
m.v6 = pyo.Var()
m.v7 = pyo.Var(
    units=pyo.units.m, bounds=(0, 1)
)  # Poorly scaled variable with lower bound
m.v8 = pyo.Var()  # unused variable

m.c1 = pyo.Constraint(expr=m.v1 + m.v2 == 10)  # Unit consistency issue
m.c2 = pyo.Constraint(expr=m.v3 == m.v4 + m.v5)
m.c3 = pyo.Constraint(expr=2 * m.v3 == 3 * m.v4 + 4 * m.v5 + m.v6)
m.c4 = pyo.Constraint(expr=m.v7 == 1e-8 * m.v1)  # Poorly scaled constraint

m.v4.fix(2)
m.v5.fix(2)
m.v6.fix(0)

Next, the instructions tell us to create an instance of the ``DiagnosticsToolbox`` and to pass the model we wish to examine as an argument.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Create an instance of the DiagnosticsToolbox: dt = DiagnosticsToolbox(m)
</div>

In [6]:
# Create an instance of the Diagnostics Toolbox

Finally, the instructions tell us to run the ``report_structural_issues()`` method. Structural issues represent issues that exist solely in the form of the model equations and thus do not depend on the current value of any of the variables. This is useful as it means we can check for these before we even call a solver, which can be critical as sometimes these issues will cause a solver to fail without providing a useful solution.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Call dt.report_structural_issues() in the cell below.
</div>

In [8]:
# Call the report_structural_issues() method

Looking at the output from the ``report_structural_issues()`` method, we can see that it provides a fairly short summary containing 4 sections.

1. The first section is a summary of the size of the model, indicating things like the number of variables and constraints. The size of the model is often important for judging how difficult it will be to solve, and this information can also be useful for comparison to what is being sent to the solver. Most solvers will report the size of the model in their output logs, and if there is a difference between what is reported here and by the solver, then you should probably look into what is happening. This section also notes some things such as if you have any deactivated Blocks, Constraints or Objectives, or if you have variables which appear in the constraints that are not part of the model; these are not necessarily wrong but it is easy to have accidentally deactivated something you did not intend to so you should always check to see that these are expected.

2. The second section provides a summary of any critical structural issues that were found - in this case we can see that there are 2 warnings we are going to need to look into. Warnings represent issues that need to be addressed before moving on as these will likely cause the solver to fail or give an incorrect answer.

3. The third section lists a summary of any cautions that are found. Cautions represent issues that may or may not be problematic; in many cases these might be expected behaviors or borderline issues. However, these could also represent conceptual issues that should be addressed, so users should take the time to investigate these and determine if they need to be fixed or not.

4. Finally, there is a section that suggests the next steps to take to help guide you through the model diagnosis process. If any warnings were identified, this section will list methods that can help you get more information on each specific problem, and if no warnings are found then it will guide you onto the next step in the model diagnosis workflow.

**Note:** there are methods available to help investigate cautions as well, but these will not show up in the next steps in order to avoid cluttering the output. You can get more information on the available methods for investigating cautions via the documentation or ``help()`` function.

In our current model, we have 2 critical issues (warnings) that we need to look into and resolve. The order in which we resolve these will generally not matter, but be aware that these can often be interrelated - fixing one warning might resolve other warnings as well (or create new ones), and sometimes you will need to look at multiple issues together to find the overall root cause.

To start with, let us look at the unit consistency issue. From the "Next Steps" section above, the toolbox is suggesting we run the ``display_components_with_inconsistent_units()`` method for more information.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Call the `display_components_with_inconsistent_units()` method from the DiagnosticsToolbox to see more information on which constraint is causing the unit consistency issues.
</div>

In [10]:
# Call the display_components_with_inconsistent_units() method

This tells us that the issue lies in constraint ``c1``. If we go back and look at this constraint, we can see that it says ``v1 + v2 == 10``. ``v1`` and ``v2`` both have units of ``m`` which is consistent, but the constant in the expression (right hand side) is unitless. Thus, we need to correct this so that the right hand side has units for the constraint to be consistent.

The cell below shows how to delete a constraint and replace it with a new one with the correct units.

<div class="alert alert-block alert-warning">
<b>Warning:</b>
Deleting components can cause unexpected issues if something else in a model is using that component (e.g., deleting a variable which is used in a constraint). You should always be careful when deleting Pyomo components and make sure you only delete components that are not used elsewhere.
</div>

In [12]:
# Delete the incorrect Constraint
m.del_component(m.c1)

# Re-create the Constraint with the correct units
m.c1 = pyo.Constraint(expr=m.v1 + m.v2 == 10 * pyo.units.m)

<div class="alert alert-block alert-warning">
<b>Warning:</b>
Fixing issues in models is often an iterative process requiring trial and error. You might also have some results from a model before running the diagnostics tools and the changes you make during debugging may make it difficult to replicate those results afterwards.
    
It is strongly recommended that you keep a record of the changes you make at each step and why, along with a Git hash (or similar version control marker) corresponding to these changes. This will allow you see what changes and why, and give you a way to go back to previous iterations if the current approach does not work out. The IDAES documentation contains recommendations on how to keep and maintain a modeling logbook.
</div>

Now, re-run the ``report_structural_issues()`` method and see if this change has fixed the unit consistency issue.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Call dt.report_structural_issues() in the cell below.
</div>

In [13]:
# Call the report_structural_issues() method

The unit consistency issue has been resolved by the changes above, so now we need to look at the structural singularity. A structural singularity occurs when one sub-part of the model is over-constrained (negative degrees of freedom), which generally means another part is under-constrained (positive degrees of freedom, assuming that there are 0 degrees of freedom overall).

The toolbox is suggesting we use the ``display_overconstrained_set()`` and ``display_underconstrained_set()`` methods to get more information on the singularity; for now, let us start with the over-constrained set.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Call dt.display_overconstrained_set() in the cell below.
</div>

In [15]:
# Call the display_overconstrained_set() method

From the output above, the toolbox is telling us that we have two constraints (``c2`` and ``c3``) which only contain a single unfixed variable (``v3``); thus in this part of the model we have -1 degree of freedom and the model is not well defined (structurally singular). If we go back and look at these constraints, we can see the that the constraints are:

``c2: v3 == v4 + v5``

``c3: 2*v3 == 3*v4 + 4*v5 + v6``

We can see that in addition to ``v3`` these constraints actually contain 3 other variables (``v4``, ``v5`` and ``v6``), however these are all variables we fixed to get our initial zero degrees of freedom. It looks like we have either accidentally fixed one too many variables or written one too many constraints.

For this example, let us assume that ``v4`` was not supposed to be fixed and unfix it.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Resolve the structural singularity and then call dt.report_structural_issues() in the cell below.
</div>

In [17]:
# Unfix v4

# Then call the report_structural_issues() method again

We can see that the over-constrained set is now empty (0 variables and 0 constraints) but the under-constrained set still has 3 variables and only 2 constraints. We can also see that there is a new warning about having 1 degree of freedom in the model, however this should not be surprising as we have just unfixed ``v4`` to resolve the over-constrained set so we have added a degree of freedom to the model.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Display the under-constrained set in the cell below.
</div>

In [19]:
# Display the under-constrained set

Looking at the output from the ``display_underconstrained_set()`` method, we can see that we have two constraints, ``c1`` and ``c4``, which contain three unfixed variables, ``v1``, ``v2`` and ``v7``. Thus, we have one degree of freedom that needs to be addressed. To fix this, we could either fix one of the variables shown or add an additional equality constraint to the model.

For this example let's fix ``v2`` to a value of 5 and then re-run the ``report_structural_issues()`` method.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Fix v2 to a value of 5 and then re-run dt.report_structural_issues.
</div>

In [21]:
# Fix v2 = 5

# Then re-run report_structural_issues() method

The toolbox is now telling us that no warnings were found, so we have resolved all the structural issues (for now at least). The toolbox is telling us that there are also 2 non-critical issues (cautions) that we should look at; one about an unused variable and one about a variable fixed to zero. If you wish, you can look into identifying and fixing these yourself, however for this example we will move on to the next step (remember that the toolbox has methods to display more details for each of these which you can find in the documentation or from the ``help()`` function).

For the Next Steps section, the toolbox is recommending we try to solve our model and then check for numerical issues.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Use the Pyomo SolverFactory to create an instance of IPOPT and then try to solve the model. Make sure to set "tee=True" as this is going to fail (and it is always good practice to review the solver logs).
</div>

In [23]:
# Create a solver object

# Try to solve the model

As hinted at above, IPOPT has returned a warning that the problem may be infeasible. Before moving on however, it is always good practice to look over the solver outputs and see what it is telling you.

<div class="alert alert-block alert-warning">
<b>Warning:</b>
A lot of useful information is contained in the solver logs which is extremely useful when diagnosing modeling issues. Each solver has its own way of reporting output and its own specific behavior, so you will need to learn to interpret the output of each solver you use. The IDAES Documentation contains some guidance on interpreting output logs for a few common solvers.
</div>

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Call the report_numerical_issues method in the cell below.
</div>

In [25]:
# Check for numerical issues

The ``report_numerical_issues()`` provides a summary similar to that which we saw for the structural issues. Firstly, it reports to us the Jacobian condition number for our problem which can give us an idea of how well-scaled the problem is, followed by  a list of warnings, cautions and suggested next steps.

Unsurprisingly, we are seeing a warning about a constraint with a large residual which we would expect when a solver reports a potentially infeasible problem. We are also seeing a warning about a variable with bound violations which might be contributing to the potential infeasibility.

For the next steps, the toolbox is suggesting some new methods to get more information on these issues; let us start by looking at the constraints with large residuals.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Display the constraint with a large residual in the cell below.
</div>

In [27]:
# Display constraint with large residual

The toolbox is telling us that the constraint which failed to converge is ``c2``, however this is generally only part of the story. Solvers work by trying to minimize the infeasibility in the model (residual of the constraints), which generally means they push any infeasibility onto the least sensitive constraint in the problem. Thus, the constraint which shows the infeasibility is often not the root cause of the problem, but only the symptom of the underlying issue.

If we look back at the constraints, we can see that the same variables also appear in ``c3`` and that some of these have bounds, all of which could be contributing to the infeasibility. In this case the solver tried to minimize the residual in all the constraints and ended up pushing all the issues off onto ``c2``.

<div class="alert alert-block alert-warning">
<b>Warning:</b>
When dealing with solver issues such as this, you should always remember that the obvious symptoms are often just the tip of the iceberg and that the real issue generally lies somewhere else; the challenge is tracing the symptoms back to their ultimate source.
</div>

Next, let us take a look at the variables at or outside their bounds as well. When a solver reports an potentially infeasible solution, the most common cause is unexpected bounds violations so you should always check these first.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Display the variables with bounds violations.
</div>

In [29]:
# Display the variables with bounds violations

The toolbox is telling us that ``v3`` is the variable with a potential issue. It is also showing us the current value and bounds for ``v3`` as well as if it is a fixed or free variable, which will be useful for diagnosing the issues.

We can see that ``v3`` is a free variable with bounds between 0 and 5 and a current value of 0. As ``v3`` is a free variable, this suggests that the solver has pushed the value to the bound where it cannot go any further, and this might be part of the cause of our infeasibility.

<div class="alert alert-block alert-warning">
<b>Warning:</b>
When dealing with bounds violations you should always start by understanding why the bounds exist and what they mean - in many cases a bound indicates the range over which the model can be trusted and that going beyond this may result in unexpected behavior due to extrapolation.
    
Never arbitrarily change a bound just because it is causing your model to be infeasible without understanding the consequences of this decision. Often, a bound violation is an indication that you need to re-think some of the constraints in your model to find alternatives which are valid in the actual range of values you are trying to solve for.
</div>

For this example, let us assume that we made a mistake with the bounds on ``v3`` and set the lower bound to be -5.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Update the bounds on v3 in the cell below.
</div>

In [31]:
# Update bounds for v3

Now that we have fixed the bounds issues, we should check whether our model is now feasible. However, before we continue we should recognize that we have just made a structural change to the model. If we were not careful, this could have introduced new structural issues to the model, so we should start from the beginning just to be sure.

<div class="alert alert-block alert-warning">
<b>Warning:</b>
In general, you should always start from the beginning of the model diagnosis workflow after you make any change to the model. Remember to also record these changes in your log book in case something unexpected happens so that you can revert any changes that cause problems.
</div>

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Check to see if there are any new structural issues in the cell below.
</div>

In [33]:
# Check for new structural issues

Our change has not introduced any new structural issues, so we can move on and try to solve the model again.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Re-solve the model in the cell below.
</div>

In [35]:
# Re-solve the model

IPOPT should have returned optimal solution now, so it looks like those bounds were what was causing the model to be infeasible. At this point, the model is now solving (for the current values at least), so you might think that the model is now ready for optimization.

However, if we look at the solver logs we can see that it took around 3 iterations for IPOPT to solve our model (depending on minor variations in computer architecture). For a model this simple, we would generally expect it to solve in only 1 iteration so there is still some room for improvement.

<div class="alert alert-block alert-warning">
<b>Warning:</b>
You should keep in mind that just because you get an optimal solution does not mean that your model is robust and free of issues.
    
You should always take the time to look over the solver logs to look for signs of trouble, even if you get an optimal solution. While you might get an optimal solution for the current state, there may be advance warning signs of issues that will cause problems later when you try to solve the model at a different state.
</div>

Let us run the ``report_numerical_issues`` method again to see if there are any other problems we need to address.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Check for additional numerical issues in the cell below.
</div>

In [37]:
# Check for additional numerical issues

The toolbox is not reporting any warnings which is good, however there are still 5 numerical cautions that it has identified which might be contributing to the larger than expected number of iterations. As mentioned earlier, the toolbox does not suggest methods for investigating these, but there are methods available. For example, we can look at the variable with an extreme value using the `display_variables_with_extreme_values()` method.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Check for additional information about variables with extreme values.
</div>

In [39]:
# Display variable with extreme value

We can see that ``v7`` is potentially causing problems due to having a very small value (on the order of magnitude of the solver tolerance). This can be especially problematic for interior point solvers like IPOPT if there is a lower bound of 0 (which there is in this case). IPOPT tries to avoid bounds and thus perturbs solutions away from these if it gets too close, which can cause convergence to be slow (or fail) if the solution lies close to the bound.

We can address this by scaling the variable so that the value of the scaled variable is large enough that the solution is not close to the lower bound. Additionally, we should look at any constraint that ``v7`` appears in (in this case ``c4``) and ensure that those constraints are well scaled as well (so that a residual of 1e-6 is reasonable for the terms involved).

For this case, we can set a scaling factor of 1e8 for both ``v7`` and ``c4`` as shown below. Note that we also need to apply Pyomo's scaling transformation to create a new scaled model to work with.

In [41]:
m.scaling_factor = pyo.Suffix(direction=pyo.Suffix.EXPORT)

m.scaling_factor[m.v7] = 1e8
m.scaling_factor[m.c4] = 1e8

scaling = pyo.TransformationFactory("core.scale_model")
scaled_model = scaling.create_using(m, rename=False)

Now that we have a scaled model, we can try to solve it and hopefully see better convergence than the unscaled model.

<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Solve the scaled model and check to see how many iterations are required.
</div>

In [42]:
# Solve scaled model

As we can see, the scaled model solved in 0 iterations (indicating that it already had the right solution). However, had we done this to the unscaled model we would have found it required 2-3 iterations again due to IPOPT perturbing the initial (correct) solution away from the bounds.

<div class="alert alert-block alert-warning">
<b>Warning:</b>
Normally in these cases we would need to map the solution from the scaled model back to the unscaled model so we can view the results. In this case, we are not actually interested in the solution so we move on with the model diagnosis.
</div>

Now that we have fixed the scaling issues, we can go back to the ``DiagnosticsToolbox`` and see if we still have any warnings. Note however that we need to look at the scaled model now rather than the original model, so we need to create a new instance of the ``DiagnosticsToolbox`` with the scaled model as the ``model`` argument.


<div class="alert alert-block alert-info">
<b>Inline Exercise:</b>
Create a new instance of the DiagnosticsToolbox and check the scaled model for issues.
</div>

In [45]:
# Create a new diagnostics toolbox for scaled model

# Report numerical issues for scaled model

We can see that applying scaling addressed two of the cautions we had before (the variable with an extreme value and an associated large value in the model Jacobian). Whilst we were able to solve the unscaled model in this case, this is in part because it was a simple linear model. In more complex, non-linear models, scaling becomes much more important and often depends strongly on the current state of the model. That is, you can often find cases where the unscaled (or poorly scaled) model solves for a limited range of conditions but fails to solve if you move too far away for the current state. Whilst you might be able to solve the model at the current state, you should always check the solver logs and numerical cautions for advanced warning signs of scaling issues that might manifest later when you try to solve the model for a different state (e.g., during optimization).

<div class="alert alert-block alert-warning">
<b>Warning:</b>
By their nature, numerical issues depend on the current values of the variables in the model, and thus may remain hidden until someone tries to solve the model close to where the issue exists. For this reason, the full model diagnostics workflow contains steps to run the numerical checks across a wide range of variable values to try to ensure that no issues remain hidden. This is beyond the scope of this tutorial however.
</div>

At this point, we have addressed all the issues that were preventing us from solving the demonstration model and so reached the end of this tutorial. For cases where we are still having trouble solving the model, we can see that the toolbox is suggesting additional methods for further debugging and these advanced features will be the focus of separate tutorials.